## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using  ExtremelyFastDecisionTreeClassifier 

## Modifications:
- - New ensemble approach (Reward Penalty based Weighted Ensemble [RPWE])


## Date: 23 Sept 2021 at 10:10 AM

In [1]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from window_slider import Slider

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [3]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [4]:
#======================================================
# ECG Feature Extraction (Wavelet Features)
#======================================================

def extract_ecg_features(raw_data):
    ch =0
    features =[]
    
    def get_features(list_values):    
        list_values = list_values
        entropy = calculate_entropy(list_values)
        crossings = calculate_crossings(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + crossings + statistics

    for i in range(raw_data.shape[0]):
            ch_data = raw_data[i]
            list_coeff = pywt.wavedec(ch_data, 'db4', level=3)
            for coeff in list_coeff:
                features += get_features(coeff)
            ch = ch+1

    return features

In [5]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [6]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [8]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((7085/51)*segment_in_sec)  #7085 is for 51 sec video record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

val_MSE = []
aro_MSE = []

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers

n_epoch =1
epochs =  1 #epoch is 1 because the model will be trained only once

optimizer= 'EFDT' #optimizer
classifier = 'EFDT_'+str(optimizer)
indx = 0
run = 1

participant = 40
videos = 16

global eeg_emotion, eda_emotion, ecg_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
ecg_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()

eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()

ecg_acc_val = metrics.Accuracy() #Accuracy
ecg_f1m_val = metrics.F1() #F1 measure  
ecg_acc_aro = metrics.Accuracy() #Accuracy
ecg_f1m_aro = metrics.F1() #F1 measure
ecg_roc_val = metrics.ROCAUC()
ecg_roc_aro = metrics.ROCAUC()
ecg_mcc_val = metrics.MCC()
ecg_mcc_aro = metrics.MCC()

 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
ecg_cm_val  = metrics.ConfusionMatrix()
ecg_cm_aro  = metrics.ConfusionMatrix()
    
itr = 0 #controls the learning rate


for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/'+str(p)+'data_eeg_AMIGOS'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/'+str(p)+'data_eda_AMIGOS.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # ECG data read from files
        ##===================================================
        def ecg_data(p):
            file_ecg = '/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/'+str(p)+'data_ecg_AMIGOS.csv'
            print(file_ecg)
            ecg_sig = pd.read_csv(file_ecg,sep=',', header = None, engine='python')
            return ecg_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_ecg = ecg_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            ecg_sig = grand_ecg.loc[grand_ecg.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            aro = eeg_sig.iloc[0,7087]
            val = eeg_sig.iloc[0,7088]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al])
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            ecg_val_prdt=[]
            ecg_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_ecg = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,14),range(2,7087)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(2,7087)]) #keeping only eda signals
            ecg_sig = np.array(ecg_sig.iloc[range(0,2),range(2,7087)]) #keeping only ecg signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_ecg.fit(ecg_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_ecg = slider_ecg.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                window_data_eeg  = np.nan_to_num(window_data_eeg) #Replacing NaN with 0 if there is such case
                features_eeg = eeg_features(window_data_eeg)
                x_eeg = np.array([features_eeg])  #EEG raw feature vector
                
#                 x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                window_data_eda  = np.nan_to_num(window_data_eda) #Replacing NaN with 0 if there is such case
                eda_features = extract_eda_features(np.array(window_data_eda))
                x_eda = np.array([eda_features]) #EDA raw feature vector
#                 x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from ECG
                #=================================================
                window_data_ecg  = np.nan_to_num(window_data_ecg) #Replacing NaN with 0 if there is such case
                ecg_features = extract_ecg_features(np.array(window_data_ecg))
                x_ecg = np.array([ecg_features]) #ECG raw feature vector
#                 x_ecg = preprocessing.normalize(ecg) #ECG normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('ecg BELT Feature shape{}:'.format(x_ecg.shape))

                    #========================
                    # For EEG data MLP model
                    #========================
                    eeg_model_val = ExtremelyFastDecisionTreeClassifier()
                    eeg_model_aro = ExtremelyFastDecisionTreeClassifier()

                    #========================
                    # For EDA data MLp model
                    #========================
                    eda_model_val = ExtremelyFastDecisionTreeClassifier()
                    eda_model_aro = ExtremelyFastDecisionTreeClassifier()
                    

                    #==============================
                    # For ecg Belt data MLP Model
                    #==============================
                    ecg_model_val = ExtremelyFastDecisionTreeClassifier()
                    ecg_model_aro = ExtremelyFastDecisionTreeClassifier()


                    init_m = init_m+1


               #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                #===========================================
                # From EEG data -- RECS System
                #===========================================

                #Valence classification EEG

                #Test the model first 
                if (indx == 0): #When the model sees the first data
                    tmp_eeg_val = 0
                    y_pred_val_eeg = 2 #other than what are class labels
                else: 
                    eeg_val = eeg_model_val.predict_proba(x_eeg)
                    tmp_eeg_val = np.max(eeg_val)
                    y_pred_val_eeg = np.argmax(eeg_val)
                    
                #Train the model once
                eeg_model_val.partial_fit(x_eeg,y_act_val)

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg)  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg)
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg)

                #Arousal classification EEG

                if (indx == 0): #When the model sees the first data
                    tmp_eeg_aro = 0
                    y_pred_aro_eeg = 2 #other than what are class labels
                else: 
                    eeg_aro = eeg_model_aro.predict_proba(x_eeg)
                    tmp_eeg_aro = np.max(eeg_aro)
                    y_pred_aro_eeg = np.argmax( eeg_aro)

                #Train the model once
                eeg_model_aro.partial_fit(x_eeg,y_act_aro)

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg)  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg)
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg)

                #===========================================
                # From EDA data 
                #===========================================

                #Valence classification EDA

                if (indx == 0): #When the model sees the first data
                    tmp_eda_val = 0
                    y_pred_val_eda = 2 #other than what are class labels
                else: 
                    eda_val = eda_model_val.predict_proba(x_eda) 
                    tmp_eda_val = np.max(eda_val)
                    y_pred_val_eda = np.argmax( eda_val)

                #Train the model once
                eda_model_val.partial_fit(x_eda,y_act_val)

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda)  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda)
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda)
                

                #Arousal classification EDA
                if (indx == 0): #When the model sees the first data
                    tmp_eda_aro = 0
                    y_pred_aro_eda = 2 #other than what are class labels
                else: 
                    eda_aro = eda_model_aro.predict_proba(x_eda)
                    tmp_eda_aro = np.max(eda_aro)
                    y_pred_aro_eda = np.argmax(eda_aro)

                #Train the model once
                eda_model_aro.partial_fit(x_eda,y_act_aro)

                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda)  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda)
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda)

                #===========================================
                # From ecg Belt data
                #===========================================

                #Valence classification ecg Belt

                if (indx == 0): #When the model sees the first data
                    tmp_ecg_val = 0
                    y_pred_val_ecg = 2 #other than what are class labels
                else: 
                    ecg_val = ecg_model_val.predict_proba(x_ecg)
                    tmp_ecg_val = np.max(ecg_val)
                    y_pred_val_ecg = np.argmax(ecg_val)

                #Train the model once
                ecg_model_val.partial_fit(x_ecg,y_act_val)

                ecg_acc_val = ecg_acc_val.update(y_act_val[0], y_pred_val_ecg)  # update the accuracy metric

                ecg_f1m_val = ecg_f1m_val.update(y_act_val[0], y_pred_val_ecg) #update f1 measure metric
                
                ecg_roc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg)
                
                ecg_mcc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg)
                

                #Arousal classification ecg Belt
                if (indx == 0): #When the model sees the first data
                    tmp_ecg_aro = 0
                    y_pred_aro_ecg = 2 #other than what are class labels
                else: 
                    ecg_aro = ecg_model_aro.predict_proba(x_ecg)
                    tmp_ecg_aro = np.max(ecg_aro)
                    y_pred_aro_ecg = np.argmax(ecg_aro)
               

                #Train the model once
                ecg_model_aro.partial_fit(x_ecg,y_act_aro)

                ecg_acc_aro = ecg_acc_aro.update(y_act_aro[0], y_pred_aro_ecg)  # update the accuracy metric

                ecg_f1m_aro = ecg_f1m_aro.update(y_act_aro[0], y_pred_aro_ecg) #update f1 measure metric
                
                ecg_roc_aro = ecg_roc_aro.update(y_act_aro[0], y_pred_aro_ecg)
                
                ecg_mcc_aro = ecg_mcc_aro.update(y_act_aro[0], y_pred_aro_ecg)
                
                
                indx = indx+1

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            ecg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_ecg_val))
            ecg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_ecg_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, ecg_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, ecg_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg)
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg)
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda)
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda)
            
            ecg_cm_val = ecg_cm_val.update(y_act_val[0], y_pred_val_ecg)
            ecg_cm_varo = ecg_cm_aro.update(y_act_aro[0], y_pred_aro_ecg)            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg, y_pred_aro_eeg]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda, y_pred_aro_eda]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_ecg, y_pred_aro_ecg]) #appending valence & arousal predicted ecg

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro = mer_cm_aro.update(y_act_aro[0], mer_aro)
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg, y_act_aro[0], y_pred_aro_eeg]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda, y_act_aro[0], y_pred_aro_eda]))
            
            ecg_emotion.append(np.array([p,v, ecg_acc_val.get(), ecg_f1m_val.get(), ecg_roc_val.get(), ecg_mcc_val.get(),
                                          ecg_acc_aro.get(),ecg_f1m_aro.get(), ecg_roc_aro.get(), ecg_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_ecg, y_act_aro[0], y_pred_aro_ecg]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
#             print('Val MSE',val_MSE)
#             print('Aro MSE',aro_MSE)
#             # print('------------------------------')
#             print('Beta_Val:', beta_val)
#             print('Beta_Aro:', beta_aro)
#             print('------------------------------')
#             print('W Val:', w_val)
#             print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('ecg BELT')

            # print("Valence Accuracy:{}".format(round(ecg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(ecg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(ecg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(ecg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(ecg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(ecg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(ecg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(ecg_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
                
                
                
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/1data_eeg_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/1data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/1data_ecg_AMIGOS.csv
Person: 1 Video:1
EEG Feature shape(1, 504):
EDA Feature shape(1, 48):
ecg BELT Feature shape(1, 96):
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:1.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:1.0
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:2
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, ar

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5
Valence F1 score:0.0
Valence ROC score:0.5
Valence MCC score:0.0
Arousal Accuracy:0.625
Arousal F1 score:0.25
Arousal ROC score:0.5333
Arousal MCC score:0.0976
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/2data_eeg_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/2data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/2data_ecg_AMIGOS.csv
Person: 2 Video:1
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
MER model
Valence Accuracy:0.5294
Valence F1 score:0.2
Valence ROC score:0.5556
Valen

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [1 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5484
Valence F1 score:0.3
Valence ROC score:0.5189
Valence MCC score:0.0476
Arousal Accuracy:0.6452
Arousal F1 score:0.1538
Arousal ROC score:0.5205
Arousal MCC score:0.0797
===============================xxxx================================
Person: 2 Video:16
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5312
Valence F1 score:0.2857
Valence ROC score:0.5118
Valence MCC score:0.0301
Arousal Accuracy:0.6562
Arousal F1 score:0.1538
Arousal ROC score:0.5216
Arousal MCC score:0.0849
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/h

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5652
Valence F1 score:0.2308
Valence ROC score:0.5173
Valence MCC score:0.051
Arousal Accuracy:0.6522
Arousal F1 score:0.1111
Arousal ROC score:0.5146
Arousal MCC score:0.0681
===============================xxxx================================
Person: 3 Video:15
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5532
Valence F1 score:0.2222
Valence ROC score:0.5137
Valence MCC score:0.0409
Arousal Accuracy:0.6383
Arousal F1 score:0.1053
Arousal ROC score:0.5127
Arousal MCC score:0.0607
===============================xxxx================================
Person: 3 Video:16
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence,

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5574
Valence F1 score:0.1818
Valence ROC score:0.5114
Valence MCC score:0.0382
Arousal Accuracy:0.5902
Arousal F1 score:0.1935
Arousal ROC score:0.5408
Arousal MCC score:0.1639
===============================xxxx================================
Person: 4 Video:14
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5484
Valence F1 score:0.1765
Valence ROC score:0.5095
Valence MCC score:0.0318
Arousal Accuracy:0.5806
Arousal F1 score:0.1875
Arousal ROC score:0.527
Arousal MCC score:0.0983
===============================xxxx================================
Person: 4 Video:15
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence,

...

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5165
Valence F1 score:0.3153
Valence ROC score:0.5074
Valence MCC score:0.0177
Arousal Accuracy:0.4974
Arousal F1 score:0.4557
Arousal ROC score:0.5039
Arousal MCC score:0.008
===============================xxxx================================
Person: 36 Video:16
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5174
Valence F1 score:0.3153
Valence ROC score:0.5077
Valence MCC score:0.0186
Arousal Accuracy:0.4965
Arousal F1 score:0.4549
Arousal ROC score:0.5028
Arousal MCC score:0.0057
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5237
Valence F1 score:0.3196
Valence ROC score:0.5124
Valence MCC score:0.0298
Arousal Accuracy:0.4966
Arousal F1 score:0.453
Arousal ROC score:0.5015
Arousal MCC score:0.0031
===============================xxxx================================
Person: 37 Video:15
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5228
Valence F1 score:0.3188
Valence ROC score:0.512
Valence MCC score:0.0289
Arousal Accuracy:0.4958
Arousal F1 score:0.4522
Arousal ROC score:0.5004
Arousal MCC score:0.0009
===============================xxxx================================
Person: 37 Video:16
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5256
Valence F1 score:0.315
Valence ROC score:0.5129
Valence MCC score:0.0314
Arousal Accuracy:0.4975
Arousal F1 score:0.4629
Arousal ROC score:0.5021
Arousal MCC score:0.0043
===============================xxxx================================
Person: 38 Video:14
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
MER model
Valence Accuracy:0.5264
Valence F1 score:0.3183
Valence ROC score:0.5143
Valence MCC score:0.0346
Arousal Accuracy:0.4967
Arousal F1 score:0.4621
Arousal ROC score:0.5011
Arousal MCC score:0.0023
===============================xxxx================================
Person: 38 Video:15
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valenc

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.5339
Valence F1 score:0.3295
Valence ROC score:0.5212
Valence MCC score:0.0512
Arousal Accuracy:0.5032
Arousal F1 score:0.4671
Arousal ROC score:0.5077
Arousal MCC score:0.0156
===============================xxxx================================
Person: 39 Video:13
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 1]
MER model
Valence Accuracy:0.533
Valence F1 score:0.3287
Valence ROC score:0.5208
Valence MCC score:0.0503
Arousal Accuracy:0.504
Arousal F1 score:0.469
Arousal ROC score:0.5086
Arousal MCC score:0.0174
===============================xxxx================================
Person: 39 Video:14
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence,

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 0]
MER model
Valence Accuracy:0.5307
Valence F1 score:0.3318
Valence ROC score:0.5182
Valence MCC score:0.0435
Arousal Accuracy:0.5008
Arousal F1 score:0.4636
Arousal ROC score:0.5054
Arousal MCC score:0.0109
===============================xxxx================================
Person: 40 Video:12
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 1, arousal: 1]
MER model
Valence Accuracy:0.5314
Valence F1 score:0.3348
Valence ROC score:0.5194
Valence MCC score:0.0464
Arousal Accuracy:0.5016
Arousal F1 score:0.4654
Arousal ROC score:0.5063
Arousal MCC score:0.0127
===============================xxxx================================
Person: 40 Video:13
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valen

In [9]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('ecg Val CM:')
print(ecg_cm_val)
print('ecg Aro CM:')
print(ecg_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    290     42
      1    228     80
EEG aro CM:
             0      1
      0    266     37
      1    235    102
EDA Val CM:
             0      1
      0    262     70
      1    229     79
EDA aro CM:
             0      1
      0    191    112
      1    193    144
ecg Val CM:
             0      1
      0    188    144
      1    155    153
ecg Aro CM:
             0      1
      0    165    138
      1    131    206
ReMECS Val CM:
             0      1
      0    263     69
      1    230     78
ReMECS aro CM:
             0      1
      0    181    122
      1    196    141


In [10]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# ecg BELT
#=============================
t_ecg_emotion =pd.DataFrame(ecg_emotion) 
t_ecg_emotion = t_ecg_emotion.iloc[:,range(2,10)]
t_ecg_emotion_avg = np.array(np.mean(t_ecg_emotion))
t_ecg_emotion_std = np.array(np.std(t_ecg_emotion))

print('ecg result Average:', t_ecg_emotion_avg)
print('ecg result std:', t_ecg_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.53821613 0.23022273 0.52369019 0.07776365 0.54842754 0.29288514
 0.54538797 0.13263733]
EEG result std: [0.02333701 0.09169264 0.02630407 0.03982341 0.05295757 0.13477343
 0.05110334 0.05796172]
EDA result Average: [0.51430927 0.30753631 0.50201012 0.0110364  0.57912001 0.50564917
 0.57155155 0.1605401 ]
EDA result std: [0.0235077  0.10306524 0.02162477 0.0259836  0.05087835 0.07711748
 0.05662635 0.08096242]
ecg result Average: [0.53572613 0.51660817 0.53068207 0.53068207 0.59118684 0.5655007
 0.58205821 0.17668028]
ecg result std: [0.02065805 0.08544078 0.02675623 0.02675623 0.05572565 0.06819721
 0.06126059 0.09224118]
MER result Average: [0.52224971 0.30220505 0.50841998 0.0242046  0.54045609 0.43327137
 0.52903939 0.07343753]
MER result std: [0.03141041 0.11948741 0.02197611 0.02637189 0.06108107 0.11152513
 0.05608822 0.08818596]


#  Result Saving all

In [11]:
time_stamp = datetime.datetime.now()
run =1
algo= 'EFDT/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/11AMIGOS_RPWE/Results_RPWE_AMIGOS/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_ecg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_ecg_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
ecg_emotion = pd.DataFrame(ecg_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
ecg_emotion.to_csv(fname_ecg)
mer_emotion.to_csv(fname_mer)


